In [2]:
//#region seen

// hide cell
enum ExprType {
    Application,
    Abstraction,
    Var,
    LiteralInt,
    LiteralBool,
}
type Expr =
    | { type: ExprType.Application, lambda: Expr, argument: Expr }
    | { type: ExprType.Abstraction, binding: Binding, body: Expr }
    | { type: ExprType.Var, name: Ident }
    | { type: ExprType.LiteralInt, value: number }
    | { type: ExprType.LiteralBool, value: boolean }
    
    
    enum TypeType {
        Int,
        Bool,
        Function,
}
type Type =
    | { type: TypeType.Int }
    | { type: TypeType.Bool }
    | { type: TypeType.Function, argumentType: Type, returnType: Type }


type Ident = string
type Binding = { name: Ident, type: Type }

// hide cell


import { Runtime } from 'jsr:@kawcco/parsebox'
import { OurModule } from "../grammar.ts"

const { Const, Tuple, Union, Ident, Ref, Array, Optional } = Runtime

const Tokens = {
    Arrow: Const('->'),
    LParen: Const('('),
    RParen: Const(')'),
    LBracket: Const('['),
    RBracket: Const(']'),
    Int: Const('int'),
    Bool: Const('bool'),
    Fn: Const('fn'),
    Comma: Const(','),
    Colon: Const(':'),
    Let: Const('let'),
    In: Const('in'),
    Equals: Const('='),
    If: Const('if'),
    Then: Const('then'),
    Else: Const('else'),
    True: Const('true'),
    False: Const('false'),
}

const Language = new OurModule({
    Expr: Tuple(
        [
            Ref<Expr>('ExprWithoutApplication'),
            Array(Tuple([
                Tokens.LParen,
                Ref<Expr>('Expr'),
                Tokens.RParen,
            ], ([, expr,]) => expr)),
        ],
        ([base, applicationArgs]) => {
            let expr = base
            for (const argument of applicationArgs) {
                expr = { type: ExprType.Application, lambda: expr, argument }
            }
            return expr
        }
    ),
    ExprWithoutApplication: Union(
        [
            Ref<Expr>('Abstraction'),
            Ref<Expr>('Int'),
            Ref<Expr>('Bool'),
            Ref<Expr>('Var'),
            Ref<Expr>('ExprParen'),
        ]
    ),
    ExprParen: Tuple(
        [
            Tokens.LParen,
            Ref<Expr>('Expr'),
            Tokens.RParen,
        ],
        ([, expr,]) => expr
    ),
    Ty: Union(
        [
            Tokens.Int,
            Tokens.Bool,
            Ref<Type>('TyFn'),
        ],
        raw => (
            raw == "int"
                ? { type: TypeType.Int }
                : raw == "bool"
                    ? { type: TypeType.Bool }
                    : raw
        ) satisfies Type,
    ),
    TyFn: Tuple(
        [
            Tokens.Fn,
            Tokens.LBracket,
            Ref<Type>('Ty'),
            Tokens.Comma,
            Ref<Type>('Ty'),
            Tokens.RBracket,
        ],
        ([, , argumentType, , returnType]) =>
            ({ type: TypeType.Function, argumentType, returnType } satisfies Type),
    ),
    Binding: Tuple(
        [
            Ident(),
            Tokens.Colon,
            Ref<Type>('Ty'),
        ],
        ([name, , type]) => ({ name, type } as Binding),
    ),
    Abstraction: Tuple(
        [
            Ref<Binding>('Binding'),
            Tokens.Arrow,
            Ref<Expr>('Expr')
        ],
        ([binding, , body]) => ({ type: ExprType.Abstraction, binding, body } satisfies Expr),
    ),
    Var: Tuple(
        [
            Ident(),
        ],
        ([name]) => ({ type: ExprType.Var, name } satisfies Expr),
    ),
    Digit: Union([
        Const("0"),
        Const("1"),
        Const("2"),
        Const("3"),
        Const("4"),
        Const("5"),
        Const("6"),
        Const("7"),
        Const("8"),
        Const("9"),
    ]),
    Int: Tuple(
        [
            Optional(Const("-")),
            Ref("Digit"),
            Array(Ref("Digit"))
        ],
        ([[minus], first_digit, digits]) => (
            {
                type: ExprType.LiteralInt,
                value: parseInt((minus ?? "") + first_digit + digits.join(""))
            } satisfies Expr
        ),
    ),
    Bool: Union(
        [
            Tokens.True,
            Tokens.False,
        ],
        raw => ({ type: ExprType.LiteralBool, value: raw == "true" } satisfies Expr),
    ),
})

//#endregion

In [ ]:
import { inspectValue } from "../inspect.ts"

import { TODO } from "../util.ts"


const typename = (v: Value) => ({ number: "int", boolean: "bool", object: "fn" } as Record<string, string>)[typeof v]
const expectInt = (v: Value) => (v.type === ValueType.Int) ? v.value : (() => { throw new Error("Runtime eval error: expected type int found " + typename(v)) })()
const expectBool = (v: Value) => (v.type === ValueType.Bool) ? v.value : (() => { throw new Error("Runtime eval error: expected type bool found " + typename(v)) })()

const T = {
    int: { type: TypeType.Int } satisfies Type,
    bool: { type: TypeType.Bool } satisfies Type,
    fn: (argumentType: Type, returnType: Type): Type =>
        ({ type: TypeType.Function, argumentType, returnType }),
} as const

const t_fn_int_bool = T.fn(T.int, T.bool)
const t_fn_int_int = T.fn(T.int, T.int)
const t_fn_bool_bool = T.fn(T.bool, T.bool)
const t_fn_int_int_int = T.fn(T.int, T.fn(T.int, T.int))
const t_fn_int_int_bool = T.fn(T.int, T.fn(T.int, T.bool))
const t_fn_bool_bool_bool = T.fn(T.bool, T.fn(T.bool, T.bool))

type JSValue = number | boolean | Value

function toValue(x: number | boolean | Value): Value {
    switch (typeof x) {
        case 'number':
            return { type: ValueType.Int, value: x }
        case 'boolean':
            return { type: ValueType.Bool, value: x }
        default:
            return x
    }
}

function impl_native<N extends string, T>(
    name: N,
    ty: (v: Value) => T,
    fn: (v: T) => JSValue,
): Value & { name: N } {
    return {
        type: ValueType.FunctionNative,
        eval: v => toValue(fn(ty(v))),
        name,
    }
}
function impl_native_2<N extends string, A, B>(
    name: N,
    ty_a: (v: Value) => A,
    ty_b: (v: Value) => B,
    fn: (a: A, b: B) => JSValue,
): Value & { name: N } {
    return {
        type: ValueType.FunctionNative,
        eval: a => ({
            type: ValueType.FunctionNative,
            eval: b => toValue(fn(ty_a(a), ty_b(b))),
            name: `${name}(${inspectValue(a as never)})`, // @fixme(scidev5) find some better way to suppress this typeerror.
        }),
        name,
    }
}

type CoreNames =
    | "odd"
    | "even"
    | "neg"
    | "add"
    | "sub"
    | "mul"
    | "eq"
    | "greater"
    | "less"
    | "not"
    | "and"
    | "nand"
    | "or"
    | "nor"
    | "xor"
    | "xnor"
/// @impl a few functions
const CORE: () => { [k in CoreNames]: [Type, Value & { name: k }] } = () => ({
    odd: [
        t_fn_int_bool,
        impl_native("odd", expectInt, v => v % 2 === 1)],
    even: TODO("implement core.even"),
    neg: [
        t_fn_int_int,
        impl_native("neg", expectInt, v => -v)],

    add: [
        t_fn_int_int_int,
        impl_native_2("add", expectInt, expectInt, (a, b) => a + b)],
    sub: [
        t_fn_int_int_int,
        impl_native_2("sub", expectInt, expectInt, (a, b) => a - b)],
    mul: [
        t_fn_int_int_int,
        impl_native_2("mul", expectInt, expectInt, (a, b) => a * b)],

    eq: [
        t_fn_int_int_bool,
        impl_native_2("eq", expectInt, expectInt, (a, b) => a === b)],
    greater: [
        t_fn_int_int_bool,
        impl_native_2("greater", expectInt, expectInt, (a, b) => a > b)],
    less: TODO("implement core.less"),

    not: TODO("implement core.not"),
    and: [
        t_fn_bool_bool_bool,
        impl_native_2("and", expectBool, expectBool, (a, b) => a && b)],
    nand: [
        t_fn_bool_bool_bool,
        impl_native_2("nand", expectBool, expectBool, (a, b) => !(a && b))],
    or: [
        t_fn_bool_bool_bool,
        impl_native_2("or", expectBool, expectBool, (a, b) => a || b)],
    nor: [
        t_fn_bool_bool_bool,
        impl_native_2("nor", expectBool, expectBool, (a, b) => !(a || b))],
    xor: [
        t_fn_bool_bool_bool,
        impl_native_2("xor", expectBool, expectBool, (a, b) => a !== b)],
    xnor: TODO("implement core.xnor"),
} as const)

const CORE_TYPES: () => Record<Ident, Type> =
    () => Object.fromEntries(Object.entries(CORE())
        .map(([name, [type, _]]) => [name, type])
    )
const CORE_VALUES: () => Record<Ident, Value> =
    () => Object.fromEntries(Object.entries(CORE())
        .map(([name, [_, value]]) => [name, value])
    )

In [ ]:
//#region seen
export enum ValueType {
    Int,
    Bool,
    Function,
    FunctionNative,
}
export type Value =
    | { type: ValueType.Int, value: number }
    | { type: ValueType.Bool, value: boolean }
    | { type: ValueType.Function, body: Expr, binding: Ident, context: Context }
    | { type: ValueType.FunctionNative, eval: (v: Value) => Value, name: string }

export type Context = Record<Ident, Value>
//#endregion seen

export function evaluate(expr: Expr, context: Context = CORE_VALUES()): Value {
    switch (expr.type) {
        case ExprType.Application: {
            const lambda = evaluate(expr.lambda, context)
            const argument = evaluate(expr.argument, context)
            if (lambda.type === ValueType.Function) {
                return evaluate(lambda.body, { ...lambda.context, [lambda.binding]: argument })
            }
            /// @impl
            TODO("implement native/core function eval")
            throw new Error("Runtime eval error: cannot call type " + ValueType[lambda.type])
        }
        //#region seen
        case ExprType.Abstraction: {
            const binding = expr.binding.name
            const body = expr.body
            return {
                type: ValueType.Function,
                binding,
                body,
                context,
            }
        }
        case ExprType.Var: {
            const value = context[expr.name]
            if (value == undefined) {
                throw new Error("Runtime eval error: variable `" + expr.name + "` not found")
            }
            return value
        }
        case ExprType.LiteralInt: {
            return { type: ValueType.Int, value: expr.value }
        }
        case ExprType.LiteralBool: {
            return { type: ValueType.Bool, value: expr.value }
        }
        //#endregion
    }
}


In [ ]:
import { inspectType } from "../inspect.ts"

export function solveTypes(expr: Expr, context: Record<string, Type> = CORE_TYPES()): Type {
    switch (expr.type) {
        case ExprType.Var: {
            if (expr.name in context) {
                return context[expr.name]
            } else {
                throw new Error(`solveTypes: variable '${expr.name}' not found`)
            }
        }
        //#region seen
        case ExprType.Application: {
            const argumentType = solveTypes(expr.argument, context)
            const lambdaType = solveTypes(expr.lambda, context)
            if (lambdaType.type !== TypeType.Function) {
                throw new Error(`solveTypes: cannot call value of type ${inspectType(lambdaType)}`)
            }
            if (!typesMatch(lambdaType.argumentType, argumentType)) {
                throw new Error(`solveTypes: lambda argument type mismatched: expected type ${inspectType(lambdaType.argumentType)}, found ${inspectType(argumentType)}`)
            }
            return lambdaType.returnType
        }
        case ExprType.Abstraction: {
            return {
                type: TypeType.Function,
                argumentType: expr.binding.type,
                returnType: solveTypes(expr.body, {
                    ...context,
                    [expr.binding.name]: expr.binding.type,
                })
            }
        }
        case ExprType.LiteralInt:
            return { type: TypeType.Int }
        case ExprType.LiteralBool:
            return { type: TypeType.Bool }
        //#endregion
    }
}

//#region seen
function typesMatch(a: Type, b: Type): boolean {
    switch (a.type) {
        case TypeType.Int:
            return b.type === TypeType.Int
        case TypeType.Bool:
            return b.type === TypeType.Bool
        case TypeType.Function: {
            if (b.type !== TypeType.Function) {
                return false
            }
            return typesMatch(a.argumentType, b.argumentType)
                && typesMatch(a.returnType, b.returnType)
        }
    }
}
//#endregion

In [ ]:
import { describe, it } from "jsr:@std/testing/bdd"
import { expect } from "jsr:@std/expect"

describe("to_value", () => {
    it("works on int", () => {
        [1, 2, 408308, 5, -2].forEach(value => {
            expect(toValue(value)).toStrictEqual({ type: ValueType.Int, value } satisfies Value)
        })
    })
    it("works on bool", () => {
        [false, true].forEach(value => {
            expect(toValue(value)).toStrictEqual({ type: ValueType.Bool, value } satisfies Value)
        })
    })
    it("works on Value", () => {
        ([
            { type: ValueType.FunctionNative, name: "id", eval: (x: Value) => x },
            { type: ValueType.Int, value: 3 },
        ] satisfies Value[]).forEach(value => {
            expect(toValue(value)).toStrictEqual(value)
        })
    })
})


describe("integration test of parse-eval", () => {
    const parse = (code: string) => Language.Parse("Expr", code) as [Expr, string] | []
    const parse_eval = (code: string) => {
        const parsed = parse(code)
        return parsed.length === 2 ? evaluate(parsed[0]) : null
    }
    it("can do basic arithmetic and logic evaluation", () => {
        const nums = [1, 2, 3, 4, 5, 6, 7, 43, 134, -123, -2, 3]
        for (const a of nums) {
            expect(parse_eval(`odd(${a})`)).toStrictEqual(parse_eval(`${a % 2 === 1}`))
            expect(parse_eval(`even(${a})`)).toStrictEqual(parse_eval(`${a % 2 === 0}`))
            expect(parse_eval(`neg(${a})`)).toStrictEqual(parse_eval(`${-a}`))
            for (const b of nums) {
                expect(parse_eval(`add(${a})(${b})`)).toStrictEqual(parse_eval(`${a + b}`))
                expect(parse_eval(`sub(${a})(${b})`)).toStrictEqual(parse_eval(`${a - b}`))
                expect(parse_eval(`mul(${a})(${b})`)).toStrictEqual(parse_eval(`${a * b}`))
                expect(parse_eval(`eq(${a})(${b})`)).toStrictEqual(parse_eval(`${a === b}`))
                expect(parse_eval(`greater(${a})(${b})`)).toStrictEqual(parse_eval(`${a > b}`))
                expect(parse_eval(`less(${a})(${b})`)).toStrictEqual(parse_eval(`${a < b}`))
            }
        }
        const bools = [false, true]
        for (const a of bools) {
            expect(parse_eval(`not(${a})`)).toStrictEqual(parse_eval(`${!a}`))
            for (const b of bools) {
                expect(parse_eval(`and(${a})(${b})`)).toStrictEqual(parse_eval(`${a && b}`))
                expect(parse_eval(`nand(${a})(${b})`)).toStrictEqual(parse_eval(`${!(a && b)}`))
                expect(parse_eval(`or(${a})(${b})`)).toStrictEqual(parse_eval(`${a || b}`))
                expect(parse_eval(`nor(${a})(${b})`)).toStrictEqual(parse_eval(`${!(a || b)}`))
                expect(parse_eval(`xor(${a})(${b})`)).toStrictEqual(parse_eval(`${a !== b}`))
                expect(parse_eval(`xnor(${a})(${b})`)).toStrictEqual(parse_eval(`${a === b}`))
            }
        }
    })
})

In [ ]:
//////////////////////////////////// REPL TIME ///////////////////////////////////////
import { repl } from "../repl.ts"

await repl(solveTypes as never, Language, evaluate as never)

> add
  = add : int -> int -> int
repl test ...
  evaluates expressions ... ok (1ms)
  rejects syntax errors ... ok (0ms)
  rejects type errors ... ok (0ms)
repl test ... ok (1ms)

ok | 1 passed (3 steps) | 0 failed (1ms)
